### 图像处理方法（Hough 直线识别）

In [ ]:
# -*- coding: UTF-8 -*-
import cv2, math
import numpy as np

DIR = 'samples\\'
ANGLE_RANGE = (-math.pi/6, math.pi/6)

# 图象预处理
img = cv2.imread( DIR + 'pic (1).jpg')
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
edges = cv2.Canny(blur,80,150,apertureSize = 3)
canny = np.uint8(np.absolute(edges))

# 直线识别
lines = cv2.HoughLinesP(edges,1,np.pi/180,threshold = 100, minLineLength = 10,maxLineGap = 100)
for line in lines:
    for x1,y1,x2,y2 in line:
        # 尝试忽略横向直线
        if abs(x1 - x2) != 0:
            slope = (y1 - y2) / (x1 - x2)
            if slope > math.tan(ANGLE_RANGE[0]) and slope < math.tan(ANGLE_RANGE[1]):
                continue
        cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
        
height, width, channels = img.shape
if height > 600:
    ratio = 600 / height
    img = cv2.resize(img, (0,0), fx=ratio, fy=ratio) 
cv2.imshow('houghlines5.jpg',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### 图片高度等比缩小至指定值 默认为600pixel

In [11]:
# -*- coding: UTF-8 -*-
import cv2
from os import listdir
from os.path import isfile, join

mypath = r'Images\004(crop)'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for file in onlyfiles:
    img = cv2.imread(mypath + '\\' +  file)
    img = cv2.resize(img, (450, 600)) 
    cv2.imwrite(mypath + '\\' +  file.split('.')[0] + '.jpg', img)

### BBox-Label -> YOLO dataset format

In [1]:
# -*- coding: UTF-8 -*-
import re
from os import listdir
from os.path import isfile, join, exists

# CHANGE TO YOUR OWN DIRECTORY
label_src = r'Labels\001'
image_src = r'Images\001'
# CHANGE TO UNIX IF YOU'RE USING MAC OR LINUX
file_path_format = 'WIN' # possible values('UNIX','WIN')

if file_path_format == 'WIN':
    slash = '\\'
elif file_path_format == 'UNIX':
    slash = '/'
else:
    raise ValueError('Invalid value for file_path_format')
    
dst = label_src + '(YOLO).txt'
fdst = open(dst, 'w')
onlyfiles = [f for f in listdir(label_src) if isfile(join(label_src, f))]

for file in onlyfiles:
    fsrc = open(label_src + '\\' + file)
    outputString = ''
    for line in fsrc.readlines():
        numbers = line.split()
        if len(numbers) == 1:
            continue
        for number in numbers:
            outputString += '%d,' % int(number)
        outputString += '%d ' % 0
    fsrc.close()
    output = image_src + '\\' + file.split('.')[0] + '.jpg ' + outputString
    fdst.write(output + '\n')
    if not re.match(r'.*(crop\d).*', file.split('.')[0]):
        for trans in ['(bright30)', '(dark30)', '(gauss)']:
            output = image_src + trans + '\\' + file.split('.')[0] + trans + '.jpg ' + outputString
            fdst.write(output + '\n')
fdst.close()

### 数据扩充（镜像、高斯模糊）

In [1]:
# -*- coding: UTF-8 -*-
import cv2, re
import numpy as np
from os import listdir, mkdir
from os.path import isfile, join, exists

# IMAGE SOURCE AND DESTINATION
src_dir = r'RealSenseCamera'

# def add_gauss_noise(image, mean=0, var=0.001):
#     image = np.array(image/255, dtype=float)
#     noise = np.random.normal(mean, var ** 0.5, image.shape)
#     out = image + noise
#     low_clip = -1. if out.min() < 0 else 0.
#     out = np.clip(out, low_clip, 1.0)
#     out = np.uint8(out*255)
#     return out

# def increase_brightness(img, value):
#     hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
#     h, s, v = cv2.split(hsv)
#     if value > 0:
#         lim = 255 - value
#         v[v > lim] = 255
#         v[v <= lim] += value
#     else:
#         v[v + value < 0] = 0
#         v[v != 0] -= value*-1
#     final_hsv = cv2.merge((h, s, v))
#     img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
#     return img

def resize_img(img, _height):
    height, width, channels = img.shape
    if height != _height:
        ratio = _height / height
        img = cv2.resize(img, (0,0), fx=ratio, fy=ratio) 
    return img

def crop_and_resize(img):
    height, width, channels = img.shape
    crop1 = img[int(height*0.2):, :int(width*0.8), :]
    crop2 = img[:int(height*0.8), int(width*0.2):, :]
    crop3 = img[int(height*0.2):, int(width*0.2):, :]
    crop4 = img[:int(height*0.8), :int(width*0.8), :]
    crop1 = resize_img(crop1, height)
    crop2 = resize_img(crop2, height)
    crop3 = resize_img(crop3, height)
    crop4 = resize_img(crop4, height)
    return crop1, crop2, crop3, crop4

def main():
    # flip
#     safe = True
#     onlyfiles = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]
#     for file in onlyfiles:
#         if not re.match(r'.*(flip).*', file):
#             img = cv2.imread(src_dir + '\\' + file)
# #             img = resize_img(img)
#             # flip
#             flip_img = cv2.flip(img, 1)
#             cv2.imwrite(src_dir + '\\' + file.split('.')[0] + '.jpg', img)
#             cv2.imwrite(src_dir + '\\' + file.split('.')[0] + '(flip).jpg', flip_img)
    # gauss, cropped, etc
    onlyfiles = [f for f in listdir(src_dir) if isfile(join(src_dir, f))]
    for file in onlyfiles:
        if re.match(r'.*\.(jpg|jpeg)', file, re.I):
            img = cv2.imread(src_dir + '\\' + file)
#             # gauss
#             gauss_img = add_gauss_noise(img)
#             cv2.imwrite(gauss_dir + '\\' + file.split('.')[0] + '(gauss).jpg', gauss_img)
#             # bright
#             bright_img = increase_brightness(img, 30)
#             cv2.imwrite(bright_dir + '\\' + file.split('.')[0] + '(bright30).jpg', bright_img)
#             # dark
#             dark_img = increase_brightness(img, -30)
#             cv2.imwrite(dark_dir + '\\' + file.split('.')[0] + '(dark30).jpg', dark_img)
            # crop
            crop_img1, crop_img2, crop_img3, crop_img4 = crop_and_resize(img)
            cv2.imwrite(src_dir + '\\' + file.split('.')[0] + 'a.jpg', crop_img1)
            cv2.imwrite(src_dir + '\\' + file.split('.')[0] + 'b.jpg', crop_img2)
            cv2.imwrite(src_dir + '\\' + file.split('.')[0] + 'c.jpg', crop_img3)
            cv2.imwrite(src_dir + '\\' + file.split('.')[0] + 'd.jpg', crop_img4)
        
if __name__ == "__main__":
    main()

### File Rename

In [9]:
# -*- coding: UTF-8 -*-
import re
from os import listdir, rename, remove
from os.path import isfile, join

i = 1
path = 'Images1.0/003'
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
for file in onlyfiles:
    if re.match(r'pic1.*', file):
        remove(path + '/' + file)

### XML processing

In [4]:
# -*- coding: UTF-8 -*-
import re, os
import xml.etree.ElementTree as ET
from os import listdir, rename, mkdir
from os.path import isfile, join, exists

src = 'Images1.4'
path = '/SomeDirectory'

onlyfiles = [f for f in listdir(src) if isfile(join(src, f))]
for file in onlyfiles:
    if not re.match(r'.*x\.xml', file, re.I):
        os.remove(src + '/' + file)
#         tree = ET.parse(src + '/' + file)
#         root = tree.getroot()
#         filename = file.split('.xml')[0] + 'x.jpg'
#         # root[1] - file name
#         root[1].text = filename
#         # root[2] - path
#         root[2].text = path + '/' + filename
#         tree.write(src + '/' + file.split('.xml')[0] + 'x.xml')
        

### Json to dataset

In [3]:
# -*- coding: UTF-8 -*-
import base64
import json
import os, re
import os.path as osp

import PIL.Image
import yaml

import utils

def draw_mask(filename):
    json_file = filename + '.json'

    data = json.load(open(json_file))
    imageData = data.get('imageData')

#     if not imageData:
#         imagePath = os.path.join(os.path.dirname(json_file), data['imagePath'])
#         with open(imagePath, 'rb') as f:
#             imageData = f.read()
#             imageData = base64.b64encode(imageData).decode('utf-8')
    img = utils.img_b64_to_arr(imageData)

    label_name_to_value = {'_background_': 0, 'p': 1, 'c': 2, 't': 3, 'o': 4, 'd':5, 's': 6}
    
#     for shape in sorted(data['shapes'], key=lambda x: x['label']):
#         label_name = shape['label']
#         if label_name in label_name_to_value:
#             label_value = label_name_to_value[label_name]
#         else:
#             label_value = len(label_name_to_value)
#             label_name_to_value[label_name] = label_value
    lbl = utils.shapes_to_label(img.shape, data['shapes'], label_name_to_value)

    label_names = [None] * (max(label_name_to_value.values()) + 1)
    for name, value in label_name_to_value.items():
        label_names[value] = name
    lbl_viz = utils.draw_label(lbl, img, label_names)

    utils.lblsave(filename + '.png', lbl)


if __name__ == '__main__':
    path = 'mask_imgs_json/'
    onlyfiles = [f for f in os.listdir(path) if osp.isfile(osp.join(path, f))]
    for file in onlyfiles:
        if not re.match(r'\..*', file, re.I):
            draw_mask(path + file.split('.')[0])